# simulation with ivs data

### Imports
Import the required libraries

In [26]:
# package(s) related to time, space and id
import datetime, time
import platform
import random
import os
import pathlib

# you need these dependencies (you can get these from anaconda)
# package(s) related to the simulation
import simpy

# spatial libraries 
import pyproj
import shapely.geometry
from shapely.geometry import Point
import shapely
import geopandas as gpd
import movingpandas as mpd

# package(s) for data handling
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pickle

# OpenTNSIM
import opentnsim
import opentnsim.core as core
import opentnsim.graph_module as graph_module
import opentnsim.plot as plot


# dtv_backend
import dtv_backend.fis as fis
import dtv_backend.network
import dtv_backend.network.network_utilities
import dtv_backend.postprocessing
import dtv_backend.simple
import dtv_backend.simulate

# Used for mathematical functions
import math             
import numpy as np

# Used for making the graph to visualize our problem
import networkx as nx  
import requests
import io
from dtv_backend.lock import Lock, Locks


### Determine situation
Definieer of er een afzetting is, en tussen welke twee nodes. (nodes moeten verbonden zijn met edge, alleen deze edge wordt verwijderd)

In [27]:
afzetting_bool = False
afzetting_node_1 = '8865412'
afzetting_node_2 = 'B20540_A'

output_dir = 'output'
dir_graphs = os.path.join(output_dir, 'graphs')


### Create graph

load locks

In [28]:
#Load lock info
url = "https://zenodo.org/records/6673604/files/FIS_locks_grouped.geojson?download=1"
resp = requests.get(url)
stream = io.BytesIO(resp.content)
locks_gdf = gpd.read_file(stream)

load graph

In [30]:
#Load graph
url = "https://zenodo.org/record/6673604/files/network_digital_twin_v0.3.pickle?download=1"
graph1 = dtv_backend.fis.load_fis_network(url).copy()

# remove edge als afzetting is True
if afzetting_bool == True:
    #remove edge
    graph1.remove_edge(afzetting_node_1, afzetting_node_2)

graph = graph_module.Graph()
graph.graph = graph1
graph.graph_info = opentnsim.utils.info(graph.graph)

save graph

In [31]:
#Save graph
os.makedirs(dir_graphs,exist_ok=True)

if afzetting_bool == True:
    # save afzetting in geopanda
    afzetting = pd.DataFrame({'A': graph1.nodes[afzetting_node_1],
    'B': graph1.nodes[afzetting_node_2]}).T
    afzetting = gpd.GeoDataFrame(afzetting)
    afzetting.to_file(os.path.join(dir_graphs, "afzetting_locatie"))

# save graph
edgelist = gpd.GeoDataFrame(nx.to_pandas_edgelist(graph.graph))
edgelist.to_file(os.path.join(dir_graphs, f"graph afzetting {afzetting_bool}"))


D:\Users\ALPF\AppData\Local\Temp\ipykernel_25344\2970863758.py:13: UserWarning:

Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.



fix cemt classes when typo

In [32]:
cemt_classes_ordered = {
            "0": 0,
            "I": 1,
            "II": 2,
            "III": 3,
            "IV": 4,
            "IVa": 5,
            "Va": 6,
            "Vb": 7,
            "VIa": 8,
            "VIb": 9,
            "VIc": 10,
            "VIIa": 11,
}
# define synonyms:
code_synonyms = {
        "_0": "0",
        "V_A": "Va",
        "V_B": "Vb",
        "VI_A": "VIa",
        "VI_B": "VIb",
        "VI_C": "VIc",
    }

for edge in graph.graph.edges:
    # replace synonyms
    if graph.graph.edges[edge]["Code"] in code_synonyms:
        graph.graph.edges[edge]["Code"] = code_synonyms[graph.graph.edges[edge]["Code"]]


        

### Make vessels and paths

##### read data

In [33]:
if os.path.isfile('data/ivs/ivs-2024-geocoded_filtered.pkl'):
    data = pickle.load(open('data/ivs/ivs-2024-geocoded_filtered.pkl', 'rb'))
else:
    # # lees data in.
    data = gpd.read_file("data\ivs\ivs-2024-geocoded.gpkg")
    data.reset_index(drop = False, inplace = True, names = 'name')

    # #filter data op bestaande iso datum en geometry.
    data['datetime'] = pd.to_datetime(data['v05_06_begindt_evenement_iso'], format = 'ISO8601', errors = 'coerce')
    data.dropna(subset = ['datetime', 'geometry'], inplace = True)
    data.to_pickle('data/ivs/ivs-2024-geocoded_filtered.pkl')


filter only on 10-12 okt and on data for which route is available.

In [34]:
import json
t_begin = pd.Timestamp('2023-10-10', tz='UTC')
t_end = pd.Timestamp('2023-10-12', tz='UTC')
failed_vessels_stremming = json.load(open(os.path.join(output_dir, f'failed_vessels_afzetting_{True}.json'), "r"))
failed_vessels_no_stremming = json.load(open(os.path.join(output_dir, f'failed_vessels_afzetting_{False}.json'), "r"))

condition_1 = data.datetime>=(t_begin)
condition_2 = data.datetime<(t_end)
condition_3 =  data.UNLO_bestemming != data.UNLO_herkomst
condition_4 = ~data.name.isin(failed_vessels_stremming)
condition_5 = ~data.name.isin(failed_vessels_no_stremming)
condition_6 = ~data.SK_CODE.isna()

idx = np.logical_and.reduce([
 condition_1,
 condition_2,
 condition_3, 
 condition_4, 
 condition_5,
 condition_6
])
data = data[idx]
data.to_pickle('data/ivs/ivs-2024-geocoded_filtered.pkl')
len(data)

1959

add CEMT klasse

In [35]:
#rename column SK
data['SK_CODE'] = data['SK_CODE'].rename({"C3l": "C3L", 
 "C2l": "C2L",
 "B04": "BO4",
 "B03": "BO3",
 "B02": "BO2",
 "B01": "BO1"})

# add cmt classe
database_rws_cemt = pd.read_json("data\DTV_shiptypes_database.json")
dict_rws_cemt = dict(zip(database_rws_cemt['RWS-class'], database_rws_cemt['CEMT-class']))
data['CEMT'] = data['SK_CODE'].map(dict_rws_cemt)

add length and width based on rws-class

In [36]:
import dtv_backend.fis as fis
data['length'] = data['SK_CODE'].map(fis.rws_klasse_to_shiplength)
data['width'] = data['SK_CODE'].map(fis.rws_klasse_to_shipwidth)

##### Create vessels (duurt +- 30 minuten)

In [37]:
from networkx.exception import NetworkXNoPath
import json
from tqdm import tqdm

# Make a class out of mix-ins
TransportResource = type('TransportResource', 
                         (core.Identifiable, core.ContainerDependentMovable, 
                          core.HasResource, core.Routable,
                          core.VesselProperties,
                         core.ExtraMetadata), 
                         {})
#define speed: 
def compute_v_provider(v_empty, v_full):
    return lambda x: 1

# load saved routes
path_vessel_routes= os.path.join(output_dir, f'vessel_routes_afzetting_{afzetting_bool}.json')
if os.path.isfile(path_vessel_routes):
    with open(path_vessel_routes, 'r') as f:
        saved_routes = json.load(f)
else:
    saved_routes = {}

# create vessels
vessels = []
failed_vessels = []
for index, row in tqdm(data.iterrows()):
    #determine path
    try:
        if str(row.name) in saved_routes.keys():
            path = saved_routes[str(row.name)]
        else:
            point_1 = fis.find_closest_node(graph.graph, Point(row.geometry.coords[0]))
            point_2 = fis.find_closest_node(graph.graph, Point(row.geometry.coords[-1]))
            path = fis.path_restricted_to_cemt_class(graph = graph.graph, 
                                            origin = point_1[0], 
                                            destination=point_2[0], 
                                            ship_cemt_classe=f"{row['CEMT']}", 
                                            ordered_cemt_classes=cemt_classes_ordered) 
        #path = nx.dijkstra_path(graph.graph, point_1[0], point_2[0], weight=compute_weight)
        #determine capacity
        capacity = max(row.v18_Laadvermogen*1000, row.v38_Vervoerd_gewicht, 1)
        data_vessel = {"env": None,
                "name": row.name,
                "type": row['v15_1_Scheepstype_RWS'],
                "B": row['width'],
                "L": row['length'],
                "route": path,
                "geometry": Point(row.geometry.coords[0]),  # lon, lat
                "capacity": capacity,
                "v": 0.5144*8, # 8 knopen
                "compute_v": compute_v_provider(v_empty=0.5144*8, v_full=0.5144*8),
                "departure_time": pd.to_datetime(row['v05_06_begindt_evenement_iso']),
                }
        vessel = TransportResource(**data_vessel)
        vessels.append(vessel)
    except NetworkXNoPath:
        failed_vessels.append(row.name)
    except ValueError:
        failed_vessels.append(row.name)
print(f"Failed vessels: {failed_vessels}")
print(f"Number of vessels: {len(vessels)}")

# korte_route = nx.dijkstra_path(graph.graph, "8865735", "8861687")
# vessels = [TransportResource(**{
#     "env" : None,
#     "name": 'korte_route', 
#     "type": 'M6',
#     "B": 1, 
#     "L": 10,
#     "route": korte_route,
#     "geometry": Point(row.geometry.coords[0]),  # lon, lat
#     "capacity": capacity,
#     "v": 1,
#     "compute_v": compute_v_provider(v_empty=1, v_full=1),
# })]

1959it [10:01,  3.26it/s]

Failed vessels: [4828, 34855, 114672, 155482, 270117, 306480, 315806, 358394, 373676, 385275, 455872, 478386, 524797, 537147, 543262, 552411, 599606, 625095, 666045, 728447, 744608, 752349, 801025, 834895, 841851, 967812, 974041, 985731, 1080985, 1111184, 1133024, 1148324, 1223723, 1269071, 1273624, 1281495, 1324949, 1336443, 1362837, 1427633, 1439248, 1484627, 1570998, 1616600, 1772501, 1812186, 1878735, 1935693, 1938025, 1941262, 1966120, 1980115, 1982442, 1996349, 1998663, 2151866, 2206027, 2219088, 2233110, 2270005, 2290938, 2310648, 2405870, 2411921, 2451457, 2468334, 2495989, 2502900]
Number of vessels: 1891


Save vessel routes

In [38]:
path_vessel_routes= os.path.join(output_dir, f'vessel_routes_afzetting_{afzetting_bool}.json')
a = {vessel.name: vessel.route for vessel in vessels}
json.dump(a, open(path_vessel_routes, 'w'))
json.dump(failed_vessels, open(os.path.join(output_dir, f'failed_vessels_afzetting_{afzetting_bool}.json'), 'w'))

### Start simulation

In [39]:
def start(env, vessel):
    while True:
        #wait untill ship will start sailing
        time_departure = time.mktime(vessel.metadata['departure_time'].timetuple())
        if env.now>time_departure:
            print(f"Vessel {vessel.name} is starting at {time_departure} \n {vessel.metadata['departure_time']} \n current time: {env.now}")
        yield env.timeout(time_departure-env.now)

        # start sailing
        vessel.log_entry_v0("Start sailing", env.now, "", vessel.geometry)
        yield from vessel.move()
        vessel.log_entry_v0("Stop sailing", env.now, "", vessel.geometry)
        
        if vessel.geometry == nx.get_node_attributes(env.FG, "geometry")[vessel.route[-1]]:
            break

In [40]:
# Start simpy environment
simulation_start = min([vessel.metadata['departure_time'] for vessel in vessels])

env = simpy.Environment(initial_time = time.mktime(simulation_start.timetuple()))
env.epoch = time.mktime(simulation_start.timetuple())

env.FG = graph.graph



Bepaal schuttijden en maak sluis-module

In [41]:
# import pandas as pd
# from dtv_backend.lock import Lock, Locks
# import simpy
# import json
# # Open data
# passages = pd.read_csv(r"P:\1359\135911\WIP\04 Data sluispasseertijden\IVS_Sluispassages_Trans2_202310.csv", date_format='%d%b%Y:%H:%M:%S', parse_dates=['v51_Datum_tijd_invaar_groen', 'v53_Datum_tijd_uitvaar_groen'])
# # drop rows without time of invaren or uitvaren
# passages.dropna(subset = ['v51_Datum_tijd_invaar_groen', 'v53_Datum_tijd_uitvaar_groen', 'IO_NAAM'], inplace = True)
# # calculate passeertijd
# passages['passeertijd'] = passages['v53_Datum_tijd_uitvaar_groen'] - passages['v51_Datum_tijd_invaar_groen']
# # remove passeertijd of 0 or more than 60 minutes
# passages = passages[(passages.passeertijd > pd.Timedelta(0)) & (passages.passeertijd <= pd.Timedelta(minutes=60))]
# # create grouped data
# passages_samenvatting = passages.groupby(by=['IO_NAAM', "Kolknaam"])[['passeertijd']].agg(['min', 'max', 'mean', 'median']).astype('timedelta64[s]')
# # Print summary
# passages_samenvatting = passages_samenvatting.apply(lambda x: x.dt.seconds/60).round().astype(int)
# passages_samenvatting.rename(columns={'passeertijd': 'passeertijd (min)'}, inplace=True)
# passages_samenvatting.style.background_gradient(cmap='Blues', vmin=0, vmax=60)

# #save in csv
# passages_samenvatting.droplevel(0, axis=1).reset_index().to_csv('output/passeertijden/passeertijden.csv')

In [42]:
#create schuttijden dict
schuttijden = pd.read_csv('output/passeertijden/passeertijden.csv', index_col=[1,2], header=0)['mean']
schuttijden = schuttijden.rename({
    "Sluis 13": "sluis 13",
    "Sluis Amerongen": "sluis Amerongen",
    "Sluis Maasbracht": "sluis Maasbracht",
    "Sluis Schijndel": "sluis Schijndel",
    "Sluis St. Andries": "sluis St. Andries",
})
schuttijden_dict = {}
for sluis,sluiskolk  in schuttijden.index:
    if sluis not in schuttijden_dict:
        schuttijden_dict[sluis] = {}
    schuttijden_dict[sluis][sluiskolk] = schuttijden[(sluis, sluiskolk)]

#create locks
locks = Locks(env=env, schuttijden=schuttijden_dict)


In [43]:
# #create dir
# import os
# os.makedirs('output/passeertijden', exist_ok = True)
# for kolknaam in passages['Kolknaam'].unique():
#     import matplotlib.pyplot as plt
#     df = passages[passages['Kolknaam'] == kolknaam]
#     (df['passeertijd'].dt.seconds/60).plot.hist(bins = 20, alpha = 0.5, label = 'kolknaam', xlabel = 'passeertijd (minuten)', title = kolknaam)
#     plt.savefig(f'output/passeertijden/{kolknaam}.png')
#     plt.close()

In [44]:
import functools
for i, vessel in enumerate(vessels):
    # Add environment and path to the vessel
    vessel.env = env

    # add passing of a lock
    filled_pass_lock = functools.partial(locks.pass_lock, vessel=vessel)
    vessel.on_pass_edge_functions = [filled_pass_lock]

    # Add the movements of the vessel to the simulation
    env.process(start(env, vessel))

env.run(
    until=pd.Timestamp('2023-10-14 00:00:00', tz='UTC').timestamp()
)

schuttijden found for lock Julianasluis
schuttijden found for lock sluis 13
schuttijden found for lock sluis St. Andries
schuttijden found for lock Volkeraksluizen
schuttijden found for lock sluis Maasbracht
schuttijden found for lock sluis Amerongen
schuttijden found for lock Oranjesluizen
schuttijden found for lock Krammersluizen
schuttijden found for lock sluis Schijndel


### Obtain vessel log information
The cel below uses the vessel log. The core function *log_entry* is used, which takes four arguments:

- **Log.** A text to describe what is logged.
- **t.** The timestamp.
- **Value.**  The value for the log (for sailing this is the distance).
- **Geometry** The location of the vessel while loggin.

In [45]:
# save the logs and trajectories of vessels
os.makedirs(os.path.join(output_dir,'plots_routes'), exist_ok = True)

log = gpd.GeoDataFrame()
for vessel in vessels:
    if len(vessel.logbook) == 0:
        continue
    vessel_log = gpd.GeoDataFrame(vessel.logbook, geometry='Geometry')
    vessel_log['trajectory_id'] = f'vessel_{vessel.name}_trip_1'
    vessel_log['object_id'] = f'vessel_{vessel.name}'
    log = pd.concat([log, vessel_log])
mpd_log = mpd.TrajectoryCollection(log, traj_id_col='trajectory_id', obj_id_col='object_id', t='Timestamp')
mpd_log.to_line_gdf().to_file(os.path.join(output_dir,'plots_routes',f'trajectories_afzetting_{afzetting_bool}.gpkg'))

d:\Users\ALPF\AppData\Local\pypoetry\Cache\virtualenvs\dtv-backend-pvs5eNot-py3.10\lib\site-packages\movingpandas\trajectory.py:166: MissingCRSWarning:

Trajectory generated without CRS. Computations will use Euclidean distances.

d:\Users\ALPF\AppData\Local\pypoetry\Cache\virtualenvs\dtv-backend-pvs5eNot-py3.10\lib\site-packages\movingpandas\trajectory.py:166: MissingCRSWarning:

Trajectory generated without CRS. Computations will use Euclidean distances.

d:\Users\ALPF\AppData\Local\pypoetry\Cache\virtualenvs\dtv-backend-pvs5eNot-py3.10\lib\site-packages\movingpandas\trajectory.py:166: MissingCRSWarning:

Trajectory generated without CRS. Computations will use Euclidean distances.

d:\Users\ALPF\AppData\Local\pypoetry\Cache\virtualenvs\dtv-backend-pvs5eNot-py3.10\lib\site-packages\movingpandas\trajectory.py:166: MissingCRSWarning:

Trajectory generated without CRS. Computations will use Euclidean distances.

d:\Users\ALPF\AppData\Local\pypoetry\Cache\virtualenvs\dtv-backend-pvs5eNot-

In [ ]:
# Save the logs of the locks
lock_dfs = []
for id, lock_object in locks.locks_resources.items():
    logbook = pd.DataFrame(lock_object.logbook)
    for chamber in lock_object.chambers:
        logbook_chamber = pd.DataFrame(chamber.logbook)
        logbook = pd.concat([logbook, logbook_chamber], axis=0)
    if len(logbook) == 0:
        continue
    logbook = logbook.sort_values(by="Timestamp").reset_index(drop=True)
    lock_properties = pd.DataFrame(logbook["Value"].values.tolist())
    lock_df = pd.concat([logbook, lock_properties], axis=1)
    lock_df['lock_id'] = id
    lock_dfs.append(lock_df)

all_locks_df = pd.concat(lock_dfs, axis=0)
all_locks_df = all_locks_df.drop(columns=['Value'])
all_locks_df.to_csv(os.path.join(output_dir,'plots_routes',f'locks_afzetting_{afzetting_bool}.csv'))


### Visualization of path
If you get an error regarding ffmpeg use [this](https://stackoverflow.com/questions/13316397/matplotlib-animation-no-moviewriters-available) answer. You have to install ffmpeg in your Conda environment. It can be done using the following command.

```bash
#Install ffmpeg using Conda
conda install -c conda-forge ffmpeg
```

In [47]:
# visualise vessel movements based on the information included in the vessel.log
plot.vessel_kml(env, vessels, stepsize = 60)
#plot.graph_kml(env)


KeyboardInterrupt: 

In [ ]:
! start explorer .